## Spooky Spooky Exploration
By Nick Brooks

In [ ]:
# Packages
import os
import numpy as np
import pandas as pd
import nltk
import random

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from subprocess import check_output
from wordcloud import WordCloud, STOPWORDS
import seaborn as sns

# Pre-Processing
import string
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
from nltk.stem import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import *

In [ ]:
import sys
sys.version

In [ ]:
# Read Data
os.chdir(r"C:\Users\Nicol\Google Drive\Learning\Jupyter\Data\spooky")
df = pd.read_csv("train.csv", index_col="id")
test = pd.read_csv("test.csv", index_col="id")

In [ ]:
pd.set_option('max_colwidth', 500)
df.text= df.text.astype(str)
df.author = pd.Categorical(df.author)
df.iloc[:20,:]

## Pre-Processing

In [ ]:
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import *
#ps = LancasterStemmer()
ps = PorterStemmer()

tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))

def preprocessing(data):
    txt = data.str.lower().str.cat(sep=' ') #1
    words = tokenizer.tokenize(txt) #2
    words = [w for w in words if not w in stop_words] #3
    #ords = [ps.stem(w) for w in words] #4
    return words

def wordfreqviz(text, x):
    word_dist = nltk.FreqDist(text)
    top_N = x
    rslt = pd.DataFrame(word_dist.most_common(top_N),
                    columns=['Word', 'Frequency']).set_index('Word')
    matplotlib.style.use('ggplot')
    rslt.plot.bar(rot=0)

def wordfreq(text, x):
    word_dist = nltk.FreqDist(text)
    top_N = x
    rslt = pd.DataFrame(word_dist.most_common(top_N),
                    columns=['Word', 'Frequency']).set_index('Word')
    print(rslt)

# Sources
# https://www.kaggle.com/longdoan/word-cloud-with-python
# https://github.com/amueller/word_cloud/issues/134
# https://amueller.github.io/word_cloud/auto_examples/masked.html

def cloud(text, title):
    # Setting figure parameters
    mpl.rcParams['figure.figsize']=(10.0,10.0)    #(6.0,4.0)
    mpl.rcParams['font.size']=12                #10 
    mpl.rcParams['savefig.dpi']=100             #72 
    mpl.rcParams['figure.subplot.bottom']=.1 
    
    # Processing Text
    stopwords = set(STOPWORDS) # Redundant
    wordcloud = WordCloud(width=1600, height=800,
                          background_color='black',
                          stopwords=stopwords,
                         ).generate(str(text))

    print(wordcloud);
    
    # Output Visualization
    fig = plt.figure(figsize=(20,10), facecolor='k')
    plt.title(title)
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.tight_layout(pad=0)

    plt.show();
    #fig.savefig("wordcloud.png", dpi=900)

## Sentiment Analysis

Could study the word mood excuded by the various authors.

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
import matplotlib.pyplot as plt

# Pre-Processing
SIA = SentimentIntensityAnalyzer()

## WordCloud

In [ ]:
for x in ('EAP', 'HPL', 'MWS'):
    print(x)
    print(cloud(preprocessing(df[df.author == x]['text']),x))

Would be interesting to extract the various types of Parts of Speech, in order to hone in on the different choie in vocabulary

## N Grams

In [ ]:
from nltk.util import ngrams
from collections import Counter

def get_ngrams(text, n):
    n_grams = ngrams((text), n)
    return [ ' '.join(grams) for grams in n_grams]

def gramfreq(text,n,num):
    # Extracting bigrams
    result = get_ngrams(text,n)
    # Counting bigrams
    result_count = Counter(result)
    # Converting to the result to a data frame
    df = pd.DataFrame.from_dict(result_count, orient='index')
    df = df.rename(columns={'index':'words', 0:'frequency'}) # Renaming index column name
    #----output----
    print(df.sort_values(["frequency"],ascending=[0])[:num])

In [ ]:
from nltk.util import ngrams
from collections import Counter

def get_ngrams(text, n):
    n_grams = ngrams((text), n)
    return [ ' '.join(grams) for grams in n_grams]

def gramfreq(text,n,num):
    # Extracting bigrams
    result = get_ngrams(text,n)
    # Counting bigrams
    result_count = Counter(result)
    # Converting to the result to a data frame
    df = pd.DataFrame.from_dict(result_count, orient='index')
    df = df.rename(columns={'index':'words', 0:'frequency'}) # Renaming index column name
    
    return df.sort_values(["frequency"],ascending=[0])[:num]

In [ ]:
def gram_table(x, gram, length):
    out = pd.DataFrame(index=None)
    for i in gram:
        table = pd.DataFrame(gramfreq(preprocessing(df[df.author == x]['text']),i,length).reset_index())
        table.columns = ["{}-Gram".format(i),"Occurence"]
        out = pd.concat([out, table], axis=1)
    return out

In [ ]:
# try to use pd.crosstab
# https://www.kaggle.com/ash316/eda-to-prediction-dietanic

In [ ]:
gram_table(x="EAP", gram=[1,2,3,4], length=20)

In [ ]:
gram_table(x="HPL", gram=[1,2,3,4], length=20)

In [ ]:
gram_table(x="MWS", gram=[1,2,3,4], length=20)

## Naive Bayes

Generative Models based off Bayes' Rule and Conditional Probabilities

In [ ]:
print("Train Vocabulary Size: {}".format(len(nltk.FreqDist(preprocessing(df['text'])))))
print("Train Size: {}".format(len(df)))
print("Test Vocabulary Size: {}".format(len(nltk.FreqDist(preprocessing(test['text'])))))
print("Test Size: {}".format(len(test)))

In [ ]:
# Number of features
all_words = nltk.FreqDist(preprocessing(df['text'])) # Calculate word occurence from whole block of text
word_features = list(all_words.keys())[:20000] 
# Number of columns (can't exceed vocab, only shrink it) from largest to smallest

In [ ]:
# Helper Functions
# for each review, records which uniqeue words out of the whole text body are present
def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

# Function to create model features
def model_prep(state, df_in):
    df_in['tokenized'] = df_in.text.astype(str).str.lower() # turn into lower case text
    df_in['tokenized'] = df_in.apply(lambda row: tokenizer.tokenize(row['tokenized']), axis=1) # apply tokenize to each row
    df_in['tokenized'] = df_in['tokenized'].apply(lambda x: [w for w in x if not w in stop_words]) # remove stopwords from each row
    df_in['tokenized'] = df_in['tokenized'].apply(lambda x: [ps.stem(w) for w in x]) # apply stemming to each row
    if state == "Train":
        print("{} Word Features: {}".format(state, len(word_features)))
        print("All Possible words in {} set: {}".format(state, len(all_words)))
        # Bag of Words with Label
        featuresets = [(find_features(text), LABEL) for (text, LABEL) in list(zip(df_in.tokenized, (df_in.author)))]
        print("Train Set Size: {}".format(len(featuresets)))
        print("Train Set Ready")
        return featuresets, word_features
    else:
        # Bag of Words without Labels
        featuresets = [(find_features(text)) for (text) in list(df_in.tokenized)]
        print("Submission Set Size: {}".format(len(featuresets)))
        print("Submission Set Ready")
        return featuresets

In [ ]:
trainset, word_features= model_prep("Train", df_in=df)

In [ ]:
submissionset = model_prep("Test", df_in=test)

In [ ]:
training_set = trainset[:15000]
testing_set = trainset[15000:]
del trainset

## Execute Model

In [ ]:
import datetime
start = time.time()
classifier = nltk.NaiveBayesClassifier.train(training_set)
# Posterior = prior_occurence * likelihood / evidence
end = time.time()
print("Model took %0.2f seconds to train"%(end - start))

In [ ]:
# Edgar Allan Poe [EAP], Mary Shelley[MWS], and HP Lovecraft[HPL]

In [ ]:
print("Classifier Test Accuracy:",(nltk.classify.accuracy(classifier, testing_set))*100)
print(classifier.show_most_informative_features(40))

In [ ]:
classifier.labels()

In [ ]:
labels = classifier.labels()
submission = pd.DataFrame(columns=labels)
for x in submissionset:
    dist = classifier.prob_classify(x)
    submission= submission.append({labels[0]:dist.prob(labels[0]),
                                   labels[1]:dist.prob(labels[1]),
                                   labels[2]:dist.prob(labels[2])},ignore_index=True)
submission.index = test.index

In [ ]:
submission.head()

In [ ]:
submission.to_csv("naive_spooky.csv")